In [1]:
import os
import json

from langchain_openai import ChatOpenAI, OpenAI
from langchain_google_genai import GoogleGenerativeAI, ChatGoogleGenerativeAI
from langchain_google_vertexai import ChatVertexAI
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain

import hashlib

from gptcache import Cache
from gptcache.manager.factory import manager_factory
from gptcache.manager import CacheBase
from gptcache.processor.pre import get_prompt
from langchain.cache import GPTCache

from dotenv import load_dotenv
load_dotenv()

True

# Cache Test

In [2]:
def get_hashed_name(name: str):
    return hashlib.sha256(name.encode()).hexdigest()

In [3]:
cache_base = CacheBase(name='sqlite')

In [4]:
def init_gptcache(cache_obj: Cache, llm: str):
    hashed_llm = get_hashed_name(llm)
    cache_obj.init(
        pre_embedding_func=get_prompt,
        data_manager=manager_factory(manager=cache_base, max_size=10_000),
    )

In [5]:
set_llm_cache(GPTCache(init_gptcache))

In [6]:
openai_llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0, max_tokens=2048)
openai_chat = ChatOpenAI(model='gpt-3.5-turbo-1106', temperature=0, max_tokens=2048)
gemini_llm = GoogleGenerativeAI(model='gemini-pro', temperature=0, max_output_tokens=2048)
gemini_chat = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0, max_output_tokens=2048)

In [ ]:
openai_llm.invoke('Hello world')

In [ ]:
openai_chat.invoke('Hello world')

In [ ]:
gemini_llm.invoke('Hello world')

In [ ]:
gemini_chat.invoke('Hello world')

# Setup LLMs

In [ ]:
llm_providers = [
    "openai",
    "google"
]

In [ ]:
llm_model = {
    "openai": {
        "base": [
            "gpt-3.5-turbo-instruct"
        ],
        "chat": [
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-1106",
            "gpt-4-32k-0613",
            "gpt-4-0613",
            "gpt-4-32k",
            "gpt-4",
            "gpt-4-vision-preview",
            "gpt-4-1106-preview",
        ]
    },
    "google": {
        "base": [
            "gemini-pro",
            "text-bison-001",
            "text-bison-002",
        ],
        "chat": [
            "gemini-pro",
            "chat-bison-001",
            "chat-bison-002",
        ]
    }
}

In [ ]:
TEMPERATURE = 0
MAX_TOKENS = 2048
set_llm_cache(SQLiteCache())

In [ ]:
models = {}

for provider in llm_providers:
    for model_type in llm_model[provider]:
        for model in llm_model[provider][model_type]:
            print(f'{provider}-{model}')
            if provider == 'openai':
                if model_type == 'base':
                    models[model] = OpenAI(model=model, temperature=TEMPERATURE, max_tokens=MAX_TOKENS)
                elif model_type == 'chat':
                    models[model] = ChatOpenAI(model=model, temperature=TEMPERATURE, max_tokens=MAX_TOKENS)
            elif provider == 'google':
                if model_type == 'base':
                    models[model] = GoogleGenerativeAI(model=model, temperature=TEMPERATURE, max_output_tokens=MAX_TOKENS)
                elif model_type == 'chat':
                    models[f"{model} - chatmodel"] = ChatGoogleGenerativeAI(model=model, temperature=TEMPERATURE, max_output_tokens=MAX_TOKENS)


In [ ]:
models.keys()

In [ ]:
models['gemini-pro'].invoke('Hello world')

In [ ]:
models['gemini-pro - chatmodel'].invoke('Hello world')

In [ ]:
models['text-bison-001'].invoke('Hello world')

In [ ]:
models['text-bison-002'].invoke('Hello world')

In [ ]:
models['text-bison-002']

In [ ]:
from langchain_google_vertexai import ChatVertexAI, HarmCategory, HarmBlockThreshold, VertexAI

In [ ]:
prompt = PromptTemplate.from_template(
    template="""Generate {{ k }} questions that test reader comprehension of the following text.

Text: {{ text }}

Questions:""",
    template_format="jinja2"
)

In [ ]:
human = "Translate this sentence from English to French. I love programming."
messages = [HumanMessage(content=human)]

safety_settings = {
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
}

chat = ChatVertexAI(temperature=1, max_output_tokens=2048)

In [ ]:
llm = VertexAI(model='text-bison-002', temperature=1, max_output_tokens=2048)

In [ ]:
chat.generate([messages], max_output_tokens=1).generations[0][0].text

In [ ]:
chain = LLMChain(
    llm=chat,
    prompt=prompt,
)

In [ ]:
llm.invoke(prompt.format(**{"k":1, "text": "Hello world"}))

In [ ]:
chain.generate([{"k":1, "text": "Hello world"}]).generations[0][0].text